In [1]:
import os
import os.path as osp
import argparse
from datetime import date
import json
import random
import time
from pathlib import Path
import numpy as np
import numpy.linalg as LA
from tqdm import tqdm
import matplotlib as mpl
import matplotlib.pyplot as plt
import cv2
import csv
import warnings

import skimage.io
import skimage.transform

import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader

import torch

import datasets
import util.misc as utils
from config import cfg
import h5py

/home/kmuvcl/anaconda3/envs/debug/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def eul2rotm_ypr(euler):
    R_x = np.array(
        [
            [1, 0, 0],
            [0, np.cos(euler[0]), -np.sin(euler[0])],
            [0, np.sin(euler[0]), np.cos(euler[0])],
        ],
        dtype=np.float32,
    )

    R_y = np.array(
        [
            [np.cos(euler[1]), 0, np.sin(euler[1])],
            [0, 1, 0],
            [-np.sin(euler[1]), 0, np.cos(euler[1])],
        ],
        dtype=np.float32,
    )

    R_z = np.array(
        [
            [np.cos(euler[2]), -np.sin(euler[2]), 0],
            [np.sin(euler[2]), np.cos(euler[2]), 0],
            [0, 0, 1],
        ],
        dtype=np.float32,
    )

    return np.dot(R_z, np.dot(R_x, R_y))


def create_masks(image):
    masks = torch.zeros((1, height, width), dtype=torch.uint8)
    return masks


def read_line_file(filename, min_line_length=10):
    segs = []  # line segments
    # csv 파일 열어서 Line 정보 가져오기
    with open(filename, "r") as csvfile:
        csvreader = csv.reader(csvfile)
        for row in csvreader:
            segs.append([float(row[0]), float(row[1]), float(row[2]), float(row[3])])
    segs = np.array(segs, dtype=np.float32)
    lengths = LA.norm(segs[:, 2:] - segs[:, :2], axis=1)
    segs = segs[lengths > min_line_length]
    return segs

def coordinate_yup(segs,org_h,org_w):
    H = np.array([0,org_h,0,org_h])
    segs[:,1] = -segs[:,1]
    segs[:,3] = -segs[:,3]
    # segs = -segs
    return(H+segs)
    

def normalize_segs(segs, pp, rho):
    pp = np.array([pp[0], pp[1], pp[0], pp[1]], dtype=np.float32)
    return (segs - pp)/rho

def focal_length_normalize(segs):
    segs = segs/517.97
    return segs


def normalize_safe_np(v, axis=-1, eps=1e-6):
    de = LA.norm(v, axis=axis, keepdims=True)
    de = np.maximum(de, eps)
    return v / de


def segs2lines_np(segs):
    ones = np.ones(len(segs))
    ones = np.expand_dims(ones, axis=-1)
    p1 = np.concatenate([segs[:, :2], ones], axis=-1)
    p2 = np.concatenate([segs[:, 2:], ones], axis=-1)

    lines = np.cross(p1, p2)

    return normalize_safe_np(lines)



def sample_segs_np(segs, num_sample, use_prob=True):
    num_segs = len(segs)
    sampled_segs = np.zeros([num_sample, 4], dtype=np.float32)
    mask = np.zeros([num_sample, 1], dtype=np.float32)
    if num_sample > num_segs:
        sampled_segs[:num_segs] = segs
        mask[:num_segs] = np.ones([num_segs, 1], dtype=np.float32)
    # else:
    #     sampled_segs = segs[:num_sample]
    #     mask[:num_segs] = np.ones([num_sample, 1], dtype=np.float32)
    else:
        lengths = LA.norm(segs[:, 2:] - segs[:, :2], axis=-1)
        prob = lengths / np.sum(lengths)
        idxs = np.random.choice(segs.shape[0], num_sample, replace=True, p=prob)
        sampled_segs = segs[idxs]
        mask = np.ones([num_sample, 1], dtype=np.float32)
    return sampled_segs, mask

# def segs_np(segs, num_sample, use_prob=True):
#     num_segs = len(segs)
#     sampled_segs = np.zeros([num_sample, 4], dtype=np.float32)
#     mask = np.zeros([num_sample, 1], dtype=np.float32)
#     if num_sample > num_segs:
#         sampled_segs[:num_segs] = segs
#         mask[:num_segs] = np.ones([num_segs, 1], dtype=np.float32)
#     # else:
#     #     sampled_segs = segs[:num_sample]
#     #     mask[:num_segs] = np.ones([num_sample, 1], dtype=np.float32)
#     else:
#         lengths = LA.norm(segs[:, 2:] - segs[:, :2], axis=-1)
#         prob = lengths / np.sum(lengths)
#         idxs = np.random.choice(segs.shape[0], num_sample, replace=True, p=prob)
#         sampled_segs = segs[idxs]
#         mask = np.ones([num_sample, 1], dtype=np.float32)
#     return sampled_segs, mask

def straight_line(segs):

    # for i in range(len(segs)):
    #     seg = np.vstack([segs[i,2]-segs[i,0],segs[i,3]-segs[i,1]])
    num_line = segs.shape[0]
    seg = np.zeros([num_line,2])
    for i in range(len(segs)):    
        seg[i,0] = segs[i,2]-segs[i,0]
        seg[i,1] = segs[i,3]-segs[i,1]
    return seg


def sample_vert_segs_np(segs, thresh_theta=22.5):
    lines = segs2lines_np(segs)
    (a, b) = lines[:, 0], lines[:, 1]
    theta = np.arctan2(np.abs(b), np.abs(a))
    thresh_theta = np.radians(thresh_theta)
    return segs[theta < thresh_theta]

def compute_vp_similarity(lines,vp):
    lines = torch.tensor(lines)
    vp = torch.tensor(vp)
    cos_sim = TF.cosine_similarity(lines, vp.unsqueeze(0), dim=-1).abs()
    
    return cos_sim

In [3]:
def load_h5py_to_dict(file_path):
        with h5py.File(file_path, 'r') as f:
            return {key: torch.tensor(f[key][:]) for key in f.keys()}

In [4]:
img = "/home/kmuvcl/dataset/data/scannet-vp/scene0770_00/frame-000080-color.png"
h5_path = "/home/kmuvcl/dataset/data/scannet-vp/scene0770_00/frame-000080-color_line.h5py"
npz_file = "/home/kmuvcl/dataset/data/scannet-vp/scene0770_00/frame-000080-vanish.npz"

In [28]:
with np.load(npz_file) as npz:
    vpts = np.array([npz[d] for d in ["x", "y", "z"]])
    print(npz["y"])
    print(npz["x"])
    print(npz["z"])
    
vpts[:, 1] *= -1
vpts /= LA.norm(vpts, axis=1, keepdims=True)

[-0.99833008 -0.01958267  0.05435465]
[ 0.05756593 -0.25730365  0.96461478]
[-0.00490409  0.96613252  0.25800064]


In [29]:
vpts = torch.tensor(vpts)

In [30]:
vpts

tensor([[ 0.0576,  0.2573,  0.9646],
        [-0.9983,  0.0196,  0.0544],
        [-0.0049, -0.9661,  0.2580]], dtype=torch.float64)

In [31]:
torch.dot(vpts[1],vpts[2])

tensor(-1.8234e-08, dtype=torch.float64)

In [50]:
image = skimage.io.imread(img)[:, :, 0:3]

In [51]:
image.shape

(512, 512, 3)

In [52]:
dic = load_h5py_to_dict(h5_path)

In [53]:
keylines = dic['klines']
num_segs = keylines.shape[0]
print(num_segs)

187


In [54]:
org_segs = np.copy(keylines.reshape(num_segs,-1).numpy())

In [55]:
org_segs.shape

(187, 4)

In [56]:
image.shape

(512, 512, 3)

In [57]:
plt.figure(figsize=(5,5))
plt.imshow(image, extent=[0, 512, 512, 0])
for i in range(num_segs):
    plt.plot(
        (org_segs[i, 0], org_segs[i, 2]),
        (org_segs[i, 1], org_segs[i, 3]),
        c="r",
        alpha=1.0,
    )                                 
plt.xlim(0, 512)
plt.ylim(512, 0)
plt.axis('off')
plt.savefig('scannet.jpg',  
            pad_inches=0, bbox_inches='tight')
plt.close('all')

In [58]:
focal_length = 2.408333333333333 * 256

In [59]:
vpts_x = vpts[0]
vpts_y = vpts[1]
vpts_z = vpts[2]

In [60]:
vpts_x = vpts_x#/vpts_x[2] * focal_length
vpts_y = vpts_y#/vpts_y[2] * focal_length 
vpts_z = vpts_z#/vpts_z[2] * focal_length

In [61]:
vpts_z

array([-0.00490409, -0.96613252,  0.25800064])

In [62]:
plt.figure(figsize=(5,5))
plt.imshow(image, extent=[-256/focal_length, 256/focal_length, 256/focal_length, -256/focal_length])
for i in range(num_segs):
    plt.plot((0, vpts_x[0]), (0, vpts_x[1]), 'r-', alpha=1.0)
    plt.plot((0, vpts_y[0]), (0, vpts_y[1]), 'g-', alpha=1.0)
    plt.plot((0, vpts_z[0]), (0, vpts_z[1]), 'b-', alpha=1.0)                               
plt.xlim(-256/focal_length, 256/focal_length)
plt.ylim(256/focal_length, -256/focal_length)
plt.axis('off')
plt.savefig('scannet_vp.jpg',  
            pad_inches=0, bbox_inches='tight')
plt.close('all')

In [63]:
org_h, org_w = image.shape[0], image.shape[1]
org_sz = np.array([org_h, org_w])
pp = (org_w / 2, org_h / 2)

In [64]:
segs = normalize_segs(org_segs, pp=pp, rho=focal_length)
sampled_segs, line_mask = sample_segs_np(segs, 512)
sampled_lines = segs2lines_np(sampled_segs)

In [65]:
print(sampled_segs.shape)

(512, 4)


In [66]:
plt.figure(figsize=(5,5))
plt.imshow(image, extent=[-256/focal_length, 256/focal_length, 256/focal_length, -256/focal_length])
for i in range(num_segs):
    plt.plot(
        (segs[i, 0], segs[i, 2]),
        (segs[i, 1], segs[i, 3]),
        c="r",
        alpha=1.0,
    )                               
plt.xlim(-256/focal_length, 256/focal_length)
plt.ylim(256/focal_length, -256/focal_length)
plt.axis('off')
plt.savefig('scannet_vp_segs.jpg',  
            pad_inches=0, bbox_inches='tight')
plt.close('all')